<a href="https://colab.research.google.com/github/Freaker99/car-detection/blob/main/car_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

drive.flush_and_unmount()
drive.mount('/content/drive')
root = '/content/drive/My Drive/car-detection/'

Mounted at /content/drive


In [ ]:
!pip install opencv-python
!pip install torch torchvision
!pip install numpy
!pip install yolov5
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
!pip install --upgrade yolov5
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt

Cloning into 'yolov5'...
remote: Enumerating objects: 15996, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 15996 (delta 100), reused 121 (delta 76), pack-reused 15831
Receiving objects: 100% (15996/15996), 14.65 MiB | 17.65 MiB/s, done.
Resolving deltas: 100% (10969/10969), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-06-22 18:51:36--  https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/264818686/56dd3480-9af3-11eb-9c92-3ecd167961dc?X-Amz-Algorithm=AWS4-HMAC-SHA256&

In [ ]:
#%cd /content/drive/My Drive/car-detection/

#!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt
#!wget https://raw.githubusercontent.com/ultralytics/yolov5/master/models/yolov5s.yaml

In [ ]:
#directory = os.getcwd()  # Pobranie ścieżki bieżącego katalogu
#files = os.listdir(directory)  # Pobranie listy plików w katalogu
#sorted_files = sorted(files, reverse=True)  # Posortowanie plików w kolejności malejącej

#latest_file_pt = "yolov5s.pt"  # Nazwa najnowszego pliku pt
#latest_file_yaml = "yolov5s.yaml"  # Nazwa najnowszego pliku yaml
#keep_files_pt = [latest_file_pt]  # Lista plików pt, które chcemy zachować
#keep_files_yaml = [latest_file_yaml]  # Lista plików yaml, które chcemy zachować

# Przechodzimy przez posortowane pliki i usuwamy wszystkie, które nie są najnowszym plikiem
#for file in sorted_files:
#    if file.startswith("yolov5s.pt.") and file != latest_file_pt:
#        os.remove(file)
#        print(f"Usunięto plik: {file}")
#    else:
#        keep_files_pt.append(file)
#    if file.startswith("yolov5s.yaml.") and file != latest_file_yaml:
#        os.remove(file)
#        print(f"Usunięto plik: {file}")
#    else:
#        keep_files_yaml.append(file)
#
#print("Zachowano następujące pliki:")
#!ls

Zachowano następujące pliki:
drive  sample_data


In [ ]:
#!git clone https://github.com/ultralytics/yolov5.git
#%cd yolov5
#from models.experimental import attempt_load
#from models.yolo import Model

#%cd /content
#!ls

#directory = os.getcwd()  # Pobranie bieżącego katalogu roboczego
#files = os.listdir(directory)  # Pobranie listy plików w katalogu

#print(files)

In [ ]:
# Importowanie niezbędnych bibliotek
import os
import torch
from torchvision.datasets import VisionDataset
from PIL import Image

# Ścieżki do folderów z obrazami
image_dir_train = 'car-detection/images/train'
image_dir_test = 'car-detection/images/val'
image_dir_val = 'car-detection/images/val'

# Ścieżki do plików etykiet
train_labels_file = root + "labels/bdd100k_labels_images_train.json"
test_labels_file = root + "labels/bdd100k_labels_images_val.json"
val_labels_file = root + "labels/bdd100k_labels_images_val.json"

In [ ]:
# Klasa datasetu BDD100K
class BDD100KDataset(VisionDataset):
    def __init__(self, labels_file, image_dir, transform=None, target_transform=None):
        super(BDD100KDataset, self).__init__(image_dir, transform=transform, target_transform=target_transform)
        self.labels = self._read_labels(labels_file)

    def __getitem__(self, index):
        image_path = os.path.join(self.root, self.images[index])
        image = Image.open(image_path).convert("RGB")
        label = self.labels[index]
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.labels)

    def _read_labels(self, labels_file):
        with open(labels_file, "r") as f:
            labels = f.read().splitlines()
        return labels

# Tworzenie instancji datasetu treningowego i walidacyjnego
train_dataset = BDD100KDataset(train_labels_file, image_dir_train)
test_dataset = BDD100KDataset(test_labels_file, image_dir_test)
val_dataset = BDD100KDataset(val_labels_file, image_dir_val)

# Definiowanie DataLoader dla treningu i walidacji
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)

In [ ]:
import torch
import torch.optim as optim
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torch.utils.data import random_split
from yolov5.models.yolo import Model


# Definicja transformacji obrazu
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Podział datasetu na treningowy i walidacyjny
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Inicjalizacja modelu YOLOv5
model = Model(root + "yolov5/models/yolov5s.yaml")

# Załadowanie wstępnie wytrenowanych wag modelu
state_dict = torch.load(root + "yolov5/yolov5s.pt")
model.load_state_dict(state_dict, strict=False)

# Ustawienie trybu trenowania modelu
model.train()

# Definicja optymalizatora
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Definicja funkcji straty
criterion = torch.nn.CrossEntropyLoss()



                 from  n    params  module                                  arguments                     
  0                -1  1      3520  yolov5.models.common.Conv               [3, 32, 6, 2, 2]              
  1                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  2                -1  1     18816  yolov5.models.common.C3                 [64, 64, 1]                   
  3                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  4                -1  2    115712  yolov5.models.common.C3                 [128, 128, 2]                 
  5                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]              
  6                -1  3    625152  yolov5.models.common.C3                 [256, 256, 3]                 
  7                -1  1   1180672  yolov5.models.common.Conv               [256, 512, 3, 2]              
  8                -1  1   1182720  

In [ ]:
weights = '/content/drive/My Drive/car-detection/yolov5s.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model = Model('/content/drive/My Drive/car-detection/yolov5s.yaml', ch=3, nc=80)
ckpt = torch.load(weights)['model'] # Load checkpoint
model.load_state_dict(ckpt.state_dict())
model.names = ckpt.names



#model.load_state_dict(torch.load('yolov5s.pt')['model'].state_dict())
#model.to(device)

# Definicja funkcji straty i optymalizatora
criterion = YOLOv5Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Pętla treningowa
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for images, targets in train_loader:
        images = images.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss}")

    # Ewaluacja na danych walidacyjnych
    model.eval()
    total_iou = 0.0
    total_samples = 0

    with torch.no_grad():
        for images, targets in val_loader:
            images = images.to(device)
            targets = targets.to(device)

            outputs = model(images)

            # Obliczanie miary IoU
            iou = calculate_iou(outputs, targets)
            total_iou += iou.sum().item()
            total_samples += images.size(0)

    avg_iou = total_iou / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Average IoU: {avg_iou}")